<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/Parallel_processing_in_Pandas_using_pandarallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3 pandas

In [39]:
import boto3
import pandas as pd

In [40]:
def get_file_list(bucket, prefix):
    s3 = boto3.client('s3',aws_access_key_id='{}',aws_secret_access_key='{}')
    keys = []
    list_objects_v2_paginator = s3.get_paginator('list_objects_v2')
    for s3_list_page in list_objects_v2_paginator.paginate(Bucket=bucket, Prefix=prefix):
        keys.extend(s3_list_page['Contents'])
    list_objects=[]
    for key in keys:
        if not key['Key'].endswith('/'):
          list_objects.append([bucket,key['Key']])
    column_names = ['input_bucket', 'input_key']
    df = pd.DataFrame(list_objects, columns=column_names)
    return df

In [ ]:
df=get_file_list('{}', '{}')
df.head()

In [42]:
len(df)

850

In [43]:
def copy_file_s3(source_bucket, source_key, destination_bucket,toggle):
    try:
      s3 = boto3.client('s3',aws_access_key_id='{}',aws_secret_access_key='{}')
      copy_source = {'Bucket': source_bucket, 'Key': source_key}
      source_file_name=source_key.split('/')[-1]
      if toggle ==0:
        destination_key=f'copy_folder/{source_file_name}'
      else:
        destination_key=f'copy_parallel/{source_file_name}'
      s3.copy_object(CopySource=copy_source, Bucket=destination_bucket, Key=destination_key)
      return "File copied successfully"
    except Exception as e:
        return f"Failed: {str(e)}"

In [44]:
import time
start_time = time.time()
df['copy_status']=df.apply(lambda row: copy_file_s3(row['input_bucket'], row['input_key'], '{}',0), axis=1)
end_time = time.time()

duration = end_time - start_time
print("Time taken:", duration, "seconds")

Time taken: 297.6148874759674 seconds


In [ ]:
df.head()

In [46]:
df[df['copy_status']!='File copied successfully'].head()

,input_bucket,input_key,copy_status


In [ ]:
!pip install pandarallel

In [48]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True,nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [49]:
df1=get_file_list('deloittesalarybucket', 'multiplesmalldataset')

In [ ]:
df1.head()

In [51]:
start_time = time.time()
df1['copy_status']=df1.parallel_apply(lambda row: copy_file_s3(row['input_bucket'], row['input_key'], '{}',1), axis=1)
end_time = time.time()

duration = end_time - start_time
print("Time taken:", duration, "seconds")

Time taken: 78.95515441894531 seconds


In [ ]:
df1.head()

In [53]:
df1[df1['copy_status']!='File copied successfully'].head()

,input_bucket,input_key,copy_status
